In [1]:
#### load packages ####
import numpy as np
import pandas as pd
import nilearn
import nibabel as nib

### Model Selection
* **Behavioral metric** : Reasoning / Spatial Orientation score
* **Morphological metric** : Sulcal depth of LPC sulci

We use a LASSO regression as a form of feature selection to determine which sulci, if any predict behaviors associated with LPC. We use cross-validation to tune the shrinking paramater (alpha) and select the model that minimizes cross-validated mean seqared error.

In [2]:
# import from sklearn CV and lasso
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from sklearn.linear_model import Lasso

loo = LeaveOneOut()

In [3]:
## Lasso ##
def lasso_(Xr, yr, alpha_vals, n_folds = loo):
    ''''
        Determine best paramters for LASSO regression and fit model
        input
        ----------
        Xr = predictors 
        yr = DV
        alpha_vals = array of possible alpha values to test 
        n_folds = folds for cross-validation. Default is loo
        Returns
        --------
        depth values as an array     
    '''

    # determine the alpha-value that minimizes MSE with GridSearchCV.
    lasso = Lasso()
    lasso_regressor = GridSearchCV(lasso, alpha_vals, scoring = 'neg_mean_squared_error', cv = n_folds)
    lasso_regressor.fit(Xr, yr)

    # best alpha and MSE
    best_alpha = lasso_regressor.best_params_
    best_MSE = lasso_regressor.best_score_
    # fit the best model
    best_model = lasso_regressor.best_estimator_
    best_model.fit(Xr,yr)
    best_model.predict(Xr)
    
    #best_model.score_
    return best_alpha, best_MSE, best_model.coef_



In [4]:
## load data (in wide format) for each hemisphere ## 

lpc_depth = pd.read_csv('lpc_sulc_depth_lasso_final')

# right
right_hemi_depth = lpc_depth[lpc_depth["hemi"] == 'rh']

# left
left_hemi_depth = lpc_depth[lpc_depth["hemi"] == 'lh']

#### right hemisphere lasso - reasoning

In [5]:
## Lasso on RH sulci ## 

## set-up model ## 

# predictors 
Xr = ['1.STS', '2.cSTS3', '3.cSTS2', '4.cSTS1',  '5.SmgS', '6.slocs_v', '10.lTOS', '11.mTOS', '12.IPS_PO', '13.pips', '14.sB', '15.aipsJ', '16.IPS', '17.SPS']

# DV 
yr = ['PMAT24_A_CR']

# alpha values
alpha = {'alpha':[.005, .01, .03, .05, .07, .09, .1, .3, .5, .7, .9]}
# model
rh_lasso = lasso_(right_hemi_depth[Xr], right_hemi_depth[yr], alpha)

## plot results ##
# alpha
print(rh_lasso[0])

# neg MSE 
print(rh_lasso[1])

# beta-vals
print(Xr)
print(rh_lasso[2])

{'alpha': 0.3}
-24.013408304498277
['1.STS', '2.cSTS3', '3.cSTS2', '4.cSTS1', '5.SmgS', '6.slocs_v', '10.lTOS', '11.mTOS', '12.IPS_PO', '13.pips', '14.sB', '15.aipsJ', '16.IPS', '17.SPS']
[ 0. -0.  0.  0.  0.  0. -0. -0. -0. -0. -0. -0. -0.  0.]


#### left hemisphere lasso - reasoning

In [6]:
## Lasso on LH sulci ## 

## set-up model ## 

#  predictors 
Xr = ['1.STS', '2.cSTS3', '3.cSTS2', '4.cSTS1',  '5.SmgS', '6.slocs_v', '10.lTOS', '11.mTOS', '12.IPS_PO', '13.pips', '14.sB', '15.aipsJ', '16.IPS', '17.SPS']

# DV 
yr = ['PMAT24_A_CR']

# alpha values
alpha = {'alpha':[.005, .01, .03, .05, .07, .09, .1, .3, .5, .7, .9]}
# model
lh_lasso = lasso_(left_hemi_depth[Xr], left_hemi_depth[yr], alpha)

# alpha
print(lh_lasso[0])

# neg MSE 
print(lh_lasso[1])

# beta-vals
print(Xr)
print(lh_lasso[2])

{'alpha': 0.3}
-24.013408304498277
['1.STS', '2.cSTS3', '3.cSTS2', '4.cSTS1', '5.SmgS', '6.slocs_v', '10.lTOS', '11.mTOS', '12.IPS_PO', '13.pips', '14.sB', '15.aipsJ', '16.IPS', '17.SPS']
[ 0. -0.  0.  0.  0.  0. -0. -0.  0.  0.  0. -0. -0.  0.]


#### right hemisphere lasso - spatial orientation

In [7]:
## Lasso on RH sulci ## 

## set-up model ## 

# predictors 
Xr = ['1.STS', '2.cSTS3', '3.cSTS2', '4.cSTS1',  '5.SmgS', '6.slocs_v', '10.lTOS', '11.mTOS', '12.IPS_PO', '13.pips', '14.sB', '15.aipsJ', '16.IPS', '17.SPS']

# DV 
yr = ['VSPLOT_TC']

# alpha values
alpha = {'alpha':[.005, .01, .03, .05, .07, .09, .1, .3, .5, .7, .9]}
# model
rh_lasso = lasso_(right_hemi_depth[Xr], right_hemi_depth[yr], alpha)

## plot results ##
# alpha
print(rh_lasso[0])

# neg MSE 
print(rh_lasso[1])

# beta-vals
print(Xr)
print(rh_lasso[2])

{'alpha': 0.3}
-26.409602076124568
['1.STS', '2.cSTS3', '3.cSTS2', '4.cSTS1', '5.SmgS', '6.slocs_v', '10.lTOS', '11.mTOS', '12.IPS_PO', '13.pips', '14.sB', '15.aipsJ', '16.IPS', '17.SPS']
[-0. -0.  0.  0. -0. -0. -0. -0.  0.  0.  0. -0. -0.  0.]


RH LPC sulci are not related to spatial orientation in young adults.

#### left hemisphere lasso - spatial orientation

In [8]:
## Lasso on LH sulci ## 

## set-up model ## 

#  predictors 
Xr = ['1.STS', '2.cSTS3', '3.cSTS2', '4.cSTS1',  '5.SmgS', '6.slocs_v', '10.lTOS', '11.mTOS', '12.IPS_PO', '13.pips', '14.sB', '15.aipsJ', '16.IPS', '17.SPS']

# DV 
yr = ['VSPLOT_TC']

# alpha values
alpha = {'alpha':[.005, .01, .03, .05, .07, .09, .1, .3, .5, .7, .9]}
# model
lh_lasso = lasso_(left_hemi_depth[Xr], left_hemi_depth[yr], alpha)

# alpha
print(lh_lasso[0])

# neg MSE 
print(lh_lasso[1])

# beta-vals
print(Xr)
print(lh_lasso[2])

{'alpha': 0.05}
-25.631911135131983
['1.STS', '2.cSTS3', '3.cSTS2', '4.cSTS1', '5.SmgS', '6.slocs_v', '10.lTOS', '11.mTOS', '12.IPS_PO', '13.pips', '14.sB', '15.aipsJ', '16.IPS', '17.SPS']
[ 0.         -9.77070812  0.         -0.          0.         -3.36292239
 -4.90838136 -0.05745792  0.          5.01816581  0.         -0.
  0.          4.30165084]


Six LH LPC sulci are related to spatial orientation performance in young adults: mTOS, iTOS, cSTS3, SLOS, pips, SPS.

### Model Evaluation
After determining which sulci are relevant for behavior, we use these sulci to construct a model to predict behavior from the selected sulci.

In [9]:
## load sklearn functions for loocv ##
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.utils import resample

#### Fit linear regression with loocv
We fit all of our models using a leave-out-out cross validation which is appropriate for small sample sizes

In [10]:
# LOOCV
def lm_loocv (Xr, yr, mod):
    """
    linear regression with a leave-one-subject out cross validation proceedure. 
    input: input: Xr = matrix or array of predictors from dataframe  in format X= df[['X1', 'X2', 'Xn']]
    yr = Array of DV of interest from dataframe in format y= df['yr']
    mod = model name as strip. will be used to save a .csv with model predictions are saved as file for later use
    returns: r_squared-cv, mean squared error, dataframe of measured and predicted y-values.

    """    
    
    # create arrays to append y-vals
    ytests = []
    ypreds = []
    
    # set input arrays as np objects
    X_array = np.array(Xr)
    y_array = np.array(yr)
 
     # split into train and test
    for train_idx, test_idx in loo.split(Xr):
        X_train, X_test = X_array[train_idx], X_array[test_idx] #requires arrays
        y_train, y_test = y_array[train_idx], y_array[test_idx]

        # model to fit
        lm = linear_model.LinearRegression() 
        
        # fit model on training data
        model = lm.fit(X_train,y_train) 
        
        # generate predictions with testing data
        y_pred = lm.predict(X_test) 
        
        #there is only one y-test and y-pred per iteration over the loo.split, 
        #so we append each score to respective lists.
        
        ytests += list(y_test) #should be your original y input
        ypreds += list(y_pred)

    #get mean squared error and R2-cv values by comparing the test to the predicted.       
    rr = metrics.r2_score(ytests, ypreds)
    ms_error = metrics.mean_squared_error(ytests, ypreds)
    
    # save predicted/measured scores as a dataframe (and write to csv)
    model_preds = pd.DataFrame({"Measured": ytests, "Predicted": ypreds})
    model_preds.to_csv(path_or_buf= "plots/" + mod + ".csv")
    
    # return regression fit metrics
    return rr , ms_error, model_preds  

#### bootrapped MSE to estimate emprical confidence intervals
loocv can result in high variance and overfitting so will will address this concern by providing empirically estimated MSE confidence intervals.

In [11]:
from sklearn.utils import resample
np.random.seed(342)

We will perform 10,000 iterations. For each iteration we will fit our loocv model on the resampled data and return the MSE. This will provide us with a range of possible MSE values which we can use to assess the variance in our model predictions. 

In [12]:
# bootstrap for selected model

def bootstrapped(data, Xr, yr, n_samples, n_iter = 10000):
    '''
    Input
        - data: dataframe
        - Xr: array of column names for x predictors in data
        - yr: column name for y predictor in data 
        - n_samples: number of samples to draw each time. Usually len(data)
        - n_iter: number of iterations. Default = 10,000
    '''
    # array to store MSE
    MSE = []
    
    # set no. iterations from input
    n_iterations = n_iter    

    for i in range(n_iterations):
        #resample data with replacement
        data_resampled = resample(data, n_samples = n_samples, replace = True)
        
        # set predictors in resampled data frame
        Xresample = data_resampled[Xr]
        yresample = data_resampled[yr]
        
        # run cross-validated linear regression on resampled data
        loocv = lm_loocv(Xresample, yresample, 'boot')
        
        # save resampled MSE
        ms_error = loocv[1]
        
        # append to MSE object
        MSE.append(ms_error)

    return MSE

In [13]:
## estimate 95% CI from bootrapped MSE ##

def est_CI(bootrapped_MSE):
    # 95% CI
    alpha = 0.95 
    
    # calc lower percentile
    p = ((1.0-alpha)/2)*100
    lower =  np.percentile(bootrapped_MSE, p)
    
    # calc upper percentile
    p = (alpha+((1.0-alpha)/2.0))*100
    upper = np.percentile(bootrapped_MSE, p)

    print ('%.1f confidence interval %.2f and %.2f' % (alpha*100, lower, upper))

#### setup data
the below will change based on our results

#### model 1a - 6 sulci ####
First we fit the model derived from the findings.

In [14]:
## set model for selected sulci ##

# x-vals are sulcal depth from sample

Xr = ['2.cSTS3', '6.slocs_v', '10.lTOS', '11.mTOS', '13.pips', '17.SPS']

# DV: spatial orientation
yr = 'VSPLOT_TC'

In [15]:
## predict score ##
mod_1a = lm_loocv(left_hemi_depth[Xr], left_hemi_depth[yr], "model_1a")

# print MSE and R2
# We can assess the fit of this model by looking at MSE-cv and R2 values
print(mod_1a[0], mod_1a[1])

0.06445202778395598 23.996481258365648


Model is slightly predictive of spatial orientation score (R2 = .04, MSE = 24.76). 
To further assess our mdoel we now bootrap the MSE (we will plot this down below)

In [22]:
# boostrap MSE
mod_1a_boot = bootstrapped(left_hemi_depth, Xr, yr, 69, n_iter = 10000) 
# 10000

In [17]:
# compute MSE  95% CI 
CI_1a = est_CI(mod_1a_boot)

95.0 confidence interval 12.46 and 49.50


In [19]:
#compute median MSE
med_1a = np.median(mod_1a_boot)
med_1a

27.40126139023092

This model offers and even stronger fit for our data (R2 = .07, MSE = 24.01)!

#### model 3 - all LH sulci
Finally, we can test a model include all LH LPC sulci in the model.

In [20]:
## set model for ##
Xr = ['1.STS', '2.cSTS3', '3.cSTS2', '4.cSTS1',  '5.SmgS', '6.slocs_v', '10.lTOS', '11.mTOS', '12.IPS_PO', '13.pips', '14.sB', '15.aipsJ', '16.IPS', '17.SPS']

In [21]:
## predict reasoning score ##
mod_3 = lm_loocv(left_hemi_depth[Xr], left_hemi_depth[yr], "model_3")

# print MSE and R2
print(mod_3[0], mod_3[1])

-0.05743246718609396 27.12277631334413


In [ ]:
# boostrap MSE
mod_3_boot = bootstrapped(left_hemi_depth, Xr, yr, 69, n_iter = 10000)

In [ ]:
# compute MSE  95% CI 
CI_3 = est_CI(mod_3_boot)

In [ ]:
#compute median MSE
med_3 = np.median(mod_3_boot)

#### Correlate model predictions
To further characterize the fit of our best model (model 1b) we can correlate our predicted scores from the model with the subjects' actual spatial orientation scores. 

In [155]:
## model 1a ## 
mod_1a_preds = mod_1a[2]
mod_1a_preds.corr(method = 'spearman')

,Measured,Predicted
Measured,1.000000,0.290021
Predicted,0.290021,1.000000


### Additional Behavior analyses
We can further evaluate our model by look at other common morphological and behavioral metrics. 
* Does cortical thickness predict spatial orientation?

we can compare the predictions from these models to our sulcal depth - spatial orientation model. 
For non-nested mdoel comparison we will use AIC

In [116]:
## AIC ##

def calculate_aic(n, mse, num_params):
    '''
    input:
    - n = num observations
    - mse =. mean squared error from regression
    - num_params = number of predictors in the model (including the intercept if applicable!)
    '''
    aic = n *np.log(mse) + 2 * num_params
    return aic

AIC is a relative measure. In order to assess the fit we will compare the AIC value of each alternative model to our best model. THe model with the lowest AIC is better. To report a difference the difference in AIC must be > 2. A difference > 10 is considered substantial. 

In [117]:
## AIC from our tertiary sulcal model 1a ##
AIC_1a = calculate_aic(27, mod_1a[1], 4)
print(AIC_1a)

93.8034945448351


#### cortical thickness
We replace depth values for each of the sulci in our selected model (1a) with cortical thickness, and then predict spatial orientation score

In [118]:
## load cortical thickness data ## 
lpc_thickness = pd.read_csv('lpc_sulc_thickness_lasso_final')
left_hemi_thickness = lpc_thickness[lpc_thickness["hemi"] == 'lh']

In [119]:
# x-vals are sulcal depth from sample
Xr = ['2.cSTS3', '6.slocs_v', '10.lTOS', '11.mTOS', '13.pips', '17.SPS']

# DV: spatial orientation
yr = 'VSPLOT_TC'

## predicting spatial orientation score from thickness ## 
model_thick_a = lm_loocv(left_hemi_thickness[Xr], left_hemi_thickness[yr], "model_thick")

print(model_thick_a[0], model_thick_a[1])

-0.01462227023780116 26.02470959817261


In [120]:
AIC_thick_a = calculate_aic(27, model_thick_a[1], 4)
print(AIC_thick_a)

95.99425430841217


In [121]:
change_AIC = AIC_thick_a - AIC_1a
print(change_AIC)

2.190759763577063


#### behavioral controls
We replace the DV for our model in 1a with other metrics LPC is implicated in 

In [122]:
# x-vals are sulcal depth from sample
Xr = ['2.cSTS3', '6.slocs_v', '10.lTOS', '11.mTOS', '13.pips', '17.SPS']

# DV: reasoning
yr = 'PMAT24_A_CR'

## predicting reasoning score from depth ## 
model_cc = lm_loocv(left_hemi_depth[Xr], left_hemi_depth[yr], "model_cc")

print(model_cc[0], model_cc[1])

-0.11575744756122974 26.02215405635451


In [107]:
AIC_cc = calculate_aic(27, model_cc[1], 4)
print(AIC_cc)

change_AIC = AIC_cc - AIC_1a
print(change_AIC)

95.99160286606582
2.1881083212307146


In [108]:
# x-vals are sulcal depth from sample
Xr = ['2.cSTS3', '6.slocs_v', '10.lTOS', '11.mTOS', '13.pips', '17.SPS']

# DV: processing speed
yr = 'ProcSpeed_Unadj'

## predicting processing speed from depth ## 
model_ps = lm_loocv(left_hemi_depth[Xr], left_hemi_depth[yr], "model_ps")

print(model_ps[0], model_ps[1])

-0.124223469893886 254.37969608588116


In [109]:
AIC_ps = calculate_aic(27, model_ps[1], 4)
print(AIC_ps)

change_AIC = AIC_ps - AIC_1a
print(change_AIC)

157.5483564670387
63.7448619222036
